# kaggle code

- (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

In [4]:
"""
A Pylearn2 Dataset class for accessing the data for the
facial expression recognition Kaggle contest for the ICML
2013 workshop on representation learning.
"""
__authors__ = "Ian Goodfellow"
__copyright__ = "Copyright 2013, Universite de Montreal"
__credits__ = ["Ian Goodfellow"]
__license__ = "3-clause BSD"
__maintainer__ = "LISA Lab"

import csv
import numpy as np
import os

from pylearn.datasets.dense_design_matrix import DefaultViewConverter
from pylearn.datasets.dense_design_matrix import DenseDesignMatrix
from pylearn.utils.string_utils import preprocess

class EmotionsDataset(DenseDesignMatrix):
    """
    A Pylearn2 Dataset class for accessing the data for the
    facial expression recognition Kaggle contest for the ICML
    2013 workshop on representation learning.
    """

    def __init__(self, which_set,
            base_path = '${PYLEARN2_DATA_PATH}/icml_2013_emotions',
            start = None,
            stop = None,
            preprocessor = None,
            fit_preprocessor = False,
            axes = ('b', 0, 1, 'c'),
            fit_test_preprocessor = False):
        """
        which_set: A string specifying which portion of the dataset
            to load. Valid values are 'train' or 'public_test'
        base_path: The directory containing the .csv files from kaggle.com.
                This directory should be writable; if the .csv files haven't
                already been converted to npy, this class will convert them
                to save memory the next time they are loaded.
        fit_preprocessor: True if the preprocessor is allowed to fit the
                   data.
        fit_test_preprocessor: If we construct a test set based on this
                    dataset, should it be allowed to fit the test set?
        """

        self.test_args = locals()
        self.test_args['which_set'] = 'public_test'
        self.test_args['fit_preprocessor'] = fit_test_preprocessor
        del self.test_args['start']
        del self.test_args['stop']
        del self.test_args['self']
        del self.test_args['__class__']

        files = {'train': 'train.csv', 'public_test' : 'test.csv'}

        try:
            filename = files[which_set]
        except KeyError:
            raise ValueError("Unrecognized dataset name: " + which_set)

        path = base_path + '/' + filename

        path = preprocess(path)

        X, y = self._load_data(path, which_set == 'train')


        if start is not None:
            assert which_set != 'test'
            assert isinstance(start, int)
            assert isinstance(stop, int)
            assert start >= 0
            assert start < stop
            assert stop <= X.shape[0]
            X = X[start:stop, :]
            if y is not None:
                y = y[start:stop, :]

        view_converter = DefaultViewConverter(shape=[48,48,1], axes=axes)

        if y is None:
            y_labels = None
        else:
            y_labels = 7
        super(EmotionsDataset, self).__init__(X=X, y=y, y_labels=y_labels, view_converter=view_converter)

        if preprocessor:
            preprocessor.apply(self, can_fit=fit_preprocessor)

    def adjust_for_viewer(self, X):
        return (X - 127.5) / 127.5

    def get_test_set(self):
        return EmotionsDataset(**self.test_args)

    def _load_data(self, path, expect_labels):

        assert path.endswith('.csv')

        # If a previous call to this method has already converted
        # the data to numpy format, load the numpy directly
        X_path = path[:-4] + '.X.npy'
        Y_path = path[:-4] + '.Y.npy'
        if os.path.exists(X_path):
            X = np.load(X_path)
            if expect_labels:
                y = np.load(Y_path)
            else:
                y = None
            return X, y

        # Convert the .csv file to numpy
        csv_file = open(path, 'r')

        reader = csv.reader(csv_file)

        # Discard header
        row = next(reader)

        y_list = []
        X_list = []

        for row in reader:
            if expect_labels:
                y_str, X_row_str = (row[0], row[1])
                y = int(y_str)
                y_list.append([y])
            else:
                X_row_str = row[1]
            X_row_strs = X_row_str.split(' ')
            X_row = [float(x) for x in X_row_strs]
            X_list.append(X_row)

        X = np.asarray(X_list).astype('float32')
        if expect_labels:
            y = np.asarray(y_list)
        else:
            y = None

        np.save(X_path, X)
        if y is not None:
            np.save(Y_path, y)

        return X, y


ModuleNotFoundError: No module named 'pylearn.datasets.dense_design_matrix'; 'pylearn.datasets' is not a package

# csv to jpg


## 1)

In [5]:
import os
import sys
import cv2
import numpy as np



In [12]:
def main():
    if len(sys.argv) < 2:
    print('Usage: python cv_to_img.py [output_path]')
return -1

output_path = sys.argv[1]`

if os.path.exists(output_path):

    os.system('rm -rf {}'.format(output_path))

os.system('mkdir {}'.format(output_path))

label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
#data = pd.read_csv('fer2013.csv', delimiter=',')
data = np.genfromtxt('fer2013.csv',delimiter=',',dtype=None)

labels = data[1:,0].astype(np.int32)
image_buffer = data[1:,1]
images = np.array([np.fromstring(image, np.uint8, sep=' ') for image in image_buffer])
usage = data[1:,2]
dataset = zip(labels, images, usage)
for i, d in enumerate(dataset):
    usage_path = os.path.join(output_path, d[-1])
    label_path = os.path.join(usage_path, label_names[d[0]])
    img = d[1].reshape((48,48))
    img_name = '%08d.jpg' % i
    img_path = os.path.join(label_path, img_name)
    if not os.path.exists(usage_path):
        os.system('mkdir {}'.format(usage_path))
    if not os.path.exists(label_path):
        os.system('mkdir {}'.format(label_path))
    cv2.imwrite(img_path, img)
    print 'Write {}'.format(img_path)



IndentationError: expected an indented block (<ipython-input-12-d934e74363e8>, line 3)

In [ ]:
if __name__ == '__main__':
    main()

## 2)

In [6]:
$mkdir fer_images

fer_data=pd.read_csv('fer2013/fer2013.csv',delimiter=',')

def save_fer_img():

for index,row in fer_data.iterrows():
pixels=np.asarray(list(row['pixels'].split(' ')),dtype=np.uint8)
img=pixels.reshape((48,48))
pathname=os.path.join('fer_images',str(index)+'.jpg')
cv2.imwrite(pathname,img)
print('image saved ias {}'.format(pathname))



SyntaxError: invalid syntax (<ipython-input-6-b16c8726284f>, line 1)

# start

## 실시간 이미지 처리 
##https://www.youtube.com/watch?v=DtBu1u5aBsc

In [19]:
# import sys
# import os
# import tensorflow
# import keras
# import numpy
import pandas as pd
import numpy as np

In [20]:
# from keras.models import Sequential
# from keras.layers import Dense,Dropout,Activation,Flatten
# from keras.layers import Conv2D,Maxpooling2D,BatchNormalization
# from keras.losses import catergoical_crossentropy
# from keras.optimiezrs import Adam
# from keras.regularizers import l2
# from keras.utils import np_utils

df = pd.read_csv('fer2013.csv')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
 Usage     35887 non-null object
 pixels    35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [21]:
 print(df.head())
    
print(df['Usage'].value_counts)

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
